In [3]:
import sys 
print(sys.version)

3.10.8 (main, Oct 13 2022, 09:48:40) [Clang 14.0.0 (clang-1400.0.29.102)]


In [6]:
# python version 3.8.6rc1
import pandas as pd
import string
import util
import io
import os
from pprint import pprint
import spacy
import pickle
import re
import numpy as np

import warnings
warnings.filterwarnings(action='once')
#warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)

import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt

import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import gensim
import gensim.corpora as corpora
from gensim.corpora import Dictionary
from gensim.models.coherencemodel import CoherenceModel
from gensim.models.ldamodel import LdaModel

import pyLDAvis
import pyLDAvis.gensim_models
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import metrics
from sklearn.decomposition import LatentDirichletAllocation

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/qiyuan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/qiyuan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/qiyuan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/qiyuan/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


ImportError: cannot import name 'entropy' from 'scipy.stats' (unknown location)

# Saving clean data in pickle

In [14]:
# comments = pd.read_csv("train.csv")
# comments = pd.read_csv("train.csv")


# comments['harmful'] = comments.iloc[:, 2:].sum(axis=1)


# df = comments[['comment_text','harmful']]

# # print merged label 
# print(df.groupby(['harmful']).count())


# df.loc[df['harmful'] != 0, 'harmful'] = 1

# # print merged label changed to ones 
# print(df.groupby(['harmful']).count())



# df_harmful = df[df['harmful'] > 0] 
# df_not_harmful = df[df['harmful'] == 0]
# # check label fraction
comments = pd.read_csv("train.csv")
comments['harmful'] = comments.iloc[:, 2:].sum(axis=1)



df = comments[['comment_text',"harmful"]]#.copy()

df.loc[df['harmful'] != 0, 'harmful'] = 1
df_harmful = df[df['harmful'] > 0] 
df_not_harmful = df[df['harmful'] == 0]
df.head()

c:\users\willi\miniconda3\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,comment_text,harmful
0,Explanation\r\nWhy the edits made under my use...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\r\nMore\r\nI can't make any real suggestions...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [15]:
print(df['harmful'].value_counts())

# Randomly select 15% of not harmful instances + all harmful instances
    #whole data result into dead kernel
    #and unbalanced data may result into potential problem 

df_not_harmful_sample = df_not_harmful.sample(frac=0.15)
sample_df = pd.concat([df_not_harmful_sample, df_harmful], axis=0)

0    143346
1     16225
Name: harmful, dtype: int64


In [16]:
sample_df

,comment_text,harmful
105447,I contest the deletion of this page because it...,0
19627,"Yes, the , and such may be better candidates....",0
77114,GOCE 2011 Year-End Report\r\n \r\nSent on beha...,0
36416,"""\r\nBut seriously\r\nI do commend you managin...",0
96583,What it means is that the description is missi...,0
...,...,...
159494,"""\r\n\r\n our previous conversation \r\n\r\nyo...",1
159514,YOU ARE A MISCHIEVIOUS PUBIC HAIR,1
159541,Your absurd edits \r\n\r\nYour absurd edits on...,1
159546,"""\r\n\r\nHey listen don't you ever!!!! Delete ...",1


In [17]:

sample_df_copy = sample_df.copy()
df_harmful_copy = df_harmful.copy()

with open('pkls/data_sampled.pkl', "wb") as f:
    pickle.dump(sample_df_copy, f)

with open('pkls/data_harmful.pkl', "wb") as f:
    pickle.dump(df_harmful_copy, f)



# Data Cleaning

In [18]:
def preprocess(df):
    def remove_punctuation(text):
        #return str(text).translate(str.maketrans('', '', string.punctuation))
        text_nopunct = "".join([char if char not in string.punctuation else ' ' for char in str(text)])
        return text_nopunct

    def tokenize_text(text):
        tokens = nltk.word_tokenize(text) 
        tokens = [token.strip() for token in tokens]
        return tokens

    def remove_stopwords(tokens):
        stopword_list = nltk.corpus.stopwords.words('english')
        filtered_tokens = [token for token in tokens if token not in stopword_list]
        #filtered_text = ' '.join(filtered_tokens)    
        return filtered_tokens

    def expand_contractions(text, contraction_mapping):

        contractions_pattern = re.compile('({})'.format('|'.join(contraction_mapping.keys())), 
                                          flags=re.IGNORECASE|re.DOTALL)
        def expand_match(contraction):
            match = contraction.group(0)
            first_char = match[0]
            expanded_contraction = contraction_mapping.get(match)\
                                    if contraction_mapping.get(match)\
                                    else contraction_mapping.get(match.lower())                       
            expanded_contraction = first_char+expanded_contraction[1:]
            return expanded_contraction

        expanded_text = contractions_pattern.sub(expand_match, text)
        expanded_text = re.sub("'", "", expanded_text)
        return expanded_text
    
    def remove_digit(text):
        text_nodigit = re.sub(r'\w*\d\w*', '',text).strip()
        return text_nodigit

    def stemming(tokenized_text):
        ps = nltk.PorterStemmer()
        stemmed = [ps.stem(word) for word in tokenized_text]
        return stemmed

    def lemmatize(tokenized_text):
        wn = nltk.WordNetLemmatizer()
        lemmatized = [wn.lemmatize(word) for word in tokenized_text]
        return lemmatized
    
    
    df["expand_contractions"] = df["comment_text"].apply(lambda x: expand_contractions(x.lower(), util.contraction_mapping))
    df["remove_punctuation"] = df["expand_contractions"].apply(lambda x: remove_punctuation(x))
    df["remove_digit"] = df["remove_punctuation"].apply(lambda x: remove_digit(x))
    df["tokenized"] = df["remove_digit"].apply(lambda x: tokenize_text(x))
    df['remove_stopwords'] =  df["tokenized"].apply(lambda x: remove_stopwords(x))
    df['stemmed'] =  df["remove_stopwords"].apply(lambda x: stemming(x))
    df['lemmatized'] =  df["stemmed"].apply(lambda x: lemmatize(x))
    return df

In [19]:
df_harmful_copy2 = df_harmful.copy()
df_harmful_preprocessed = preprocess(df_harmful_copy2)

In [20]:
df_sampled_preprocessed = preprocess(sample_df)

In [21]:
# pickle.dump(df_sampled_preprocessed, open('pkls/data_sampled_processed.pkl', 'wb'))
# pickle.dump(df_harmful_preprocessed, open('pkls/data_harmful_processed.pkl', 'wb'))

with open('pkls/data_sampled_processed.pkl', "wb") as f:
    pickle.dump(df_sampled_preprocessed, f)

with open('pkls/data_harmful_processed.pkl', "wb") as f:
    pickle.dump(df_harmful_preprocessed, f)
